In [1]:
!pip install keras==2.8

  Using cached keras-2.8.0-py2.py3-none-any.whl (1.4 MB)
  Attempting uninstall: keras
    Found existing installation: keras 2.9.0
    Uninstalling keras-2.9.0:
      Successfully uninstalled keras-2.9.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.9.1 requires keras<2.10.0,>=2.9.0rc0, but you have keras 2.8.0 which is incompatible.


In [2]:
!pip install api

In [3]:
!pip install keras_vggface

In [4]:
#!pip install tensorflow-cpu

In [5]:
!pip install tensor


In [6]:
pip install tensorflow


  Using cached keras-2.9.0-py2.py3-none-any.whl (1.6 MB)
  Attempting uninstall: keras
    Found existing installation: keras 2.8.0
    Uninstalling keras-2.8.0:
      Successfully uninstalled keras-2.8.0
Note: you may need to restart the kernel to use updated packages.


In [7]:
#!pip install tensorflow-gpu 

In [8]:
!pip3 install mxnet

In [ ]:
import numpy as np
np.random.seed(1337) 
import os 
import tensorflow as tf
from keras import backend as K
from keras.models import Model, Sequential
from keras.layers import Concatenate, Embedding, Input, Dropout, Dense, Activation, Bidirectional, GlobalMaxPooling1D
from keras.layers import Convolution1D, LSTM, GRU, CuDNNGRU, CuDNNLSTM, Permute, Lambda, Flatten
from keras import optimizers
#from keras.engine.topology import Layer
#from keras.utils.layer_utils import Layer
#from keras import layers
from tensorflow.keras.layers import Layer, InputSpec
from keras import regularizers
from keras.models import load_model

#os.environ['CUDA_VISIBLE_DEVICES'] = 1 

In [ ]:
#from google.colab import drive
#drive.mount('/content/drive')

In [ ]:
data_npy_path = "/Users/shan/Desktop/Event_Extraction/data/data_all.npy"
scorer_perl_script = "/Users/shan/Desktop/Event_Extraction/Corpus/SemEval2010_task8_scorer-v1.2/semeval2010_task8_scorer-v1.2.pl"
train_answer_keys = '/Users/shan/Desktop/Event_Extraction/files/train_answer_keys.txt'
val_answer_keys = '/Users/shan/Desktop/Event_Extraction/files/val_answer_keys.txt'
test_answer_keys = '/Users/shan/Desktop/Event_Extraction/files/test_answer_keys.txt'

proposed_ans = "/Users/shan/Desktop/Event_Extraction/model/proposed_ans.txt"
scorer_output = "/Users/shan/Desktop/Event_Extraction/model/scorer_output.txt"

In [ ]:
def get_mask_entities(x):
    ''' 1 for entity words, 0 otherwise '''
    
    ret = np.zeros_like(x)
    #print("get_mask shape", ret.shape)
    # e1s 2409
    # e1e 2408
    # e2s 2411
    # e2e 2410
    for i in range(x.shape[0]): 
        e1 = [0, 0]
        e2 = [0, 0]
        for j in range(x.shape[1]):
            if x[i][j] == 2409:
                e1[0] = j
            elif x[i][j] == 2408: 
                e1[1] = j
            elif x[i][j] == 2411: 
                e2[0] = j
            elif x[i][j] == 2410: 
                e2[1] = j
                break
        for j in range(e1[0]+1, e1[1]): 
            ret[i][j] = 1
        for j in range(e2[0]+1, e2[1]): 
            ret[i][j] = 1
        #print(ret[i][e1[0]:e1[1]+1], x[i][e1[0]:e1[1]+1])
    
    return ret

In [ ]:
# Data Loading
from keras.datasets import imdb

# save np.load
# modify the default parameters of np.load
np.load.__defaults__=(None, True, True, 'ASCII')

# call load_data with allow_pickle implicitly set to true

train_set, val_set, test_set, embedding, label_to_int, int_to_label =  np.load(data_npy_path)
# restore np.load for future normal usage


train_x, train_y = train_set
val_x, val_y = val_set
test_x, test_y = test_set

train_x_copy = train_x
train_y_copy = train_y

max_sentence_len = train_x.shape[1]
n_out = len(label_to_int)

print("train_x.shape", train_x.shape)
print("train_y.shape", train_y.shape)
print("val_x.shape", val_x.shape)
print("val_y.shape", val_y.shape)
print("test_x.shape", test_x.shape)
print("test_y.shape", test_y.shape)
print("embedding.shape", embedding.shape)
print("len(label_to_int)", len(label_to_int))
print("len(int_to_label)", len(int_to_label))
print("max_sentence_len", max_sentence_len)
print("n_out", n_out)
print()

In [ ]:
train_x_mask = get_mask_entities(train_x)
train_x_mask_copy = train_x_mask
val_x_mask = get_mask_entities(val_x)
test_x_mask = get_mask_entities(test_x)

print("train_x_mask.shape", train_x_mask.shape)
print("train_x_mask_copy.shape", train_x_mask_copy.shape)
print("val_x_mask.shape", val_x_mask.shape)
print("test_x_mask.shape", test_x_mask.shape)

In [ ]:
# Parameters 

units = 64
batch_size = 128 #10
dropout_emb = 0.64
dropout_model = 0.32
dropout_pen = 0.32
l2_val = 0.00001
learning_rate = 1.0
activation_fn = 'tanh'
nb_epoch = 256
adadelta = tf.optimizers.Adadelta(lr=learning_rate, decay=0.0)
save_model = True
es_epoch_stop = 20

print("units", units)
print("batch_size", batch_size)
print("dropout_emb", dropout_emb)
print("dropout_model", dropout_model)
print("dropout_pen", dropout_pen)
print("l2_val", l2_val)
print("learning_rate", learning_rate)
print("activation_fn", activation_fn)
print("nb_epoch", nb_epoch)
print("adadelta", adadelta)
print("save_model", save_model)
print("es_epoch_stop", es_epoch_stop)
print()

embeddings_reg = regularizers.l2(l2_val)
kernel_reg = regularizers.l2(l2_val)
recurrent_reg = regularizers.l2(l2_val)

In [ ]:
class MaskMaxPoolingLayer(Layer):
    # input_shape [(None, 101, 600), (None, 101)]
    # input:  (?, 101, 600) (?, 101)
    # output: (?, 600)
    
    def __init__(self, **kwargs):
        super(MaskMaxPoolingLayer, self).__init__(**kwargs)
    
    def build(self, input_shape):
        super(MaskMaxPoolingLayer, self).build(input_shape)  # Be sure to call this somewhere!

    def call(self, x):
        x_1_float32 = K.cast(x[1], dtype='float32')
        #print("x_1_float32", x_1_float32)
        x_0 = K.permute_dimensions(x[0], pattern=[2, 0, 1])
        #print("x_0", x_0)
        x_0 = tf.multiply(x_0, x_1_float32)
        #print("x_0", x_0)
        x_0 = K.permute_dimensions(x_0, pattern=[1, 2, 0])
        #print("x_0", x_0)
        x_0 = K.max(x_0, axis=-2)
        #print("x_0", x_0)
        return x_0
    
    def compute_output_shape(self, input_shape):
        # input_shape [(None, 101, 600), (None, 101)]
        return (input_shape[0][0], input_shape[0][-1])

In [ ]:
words_input = Input(shape=(max_sentence_len,), dtype='int32', name='words_input')
words_input_mask = Input(shape=(max_sentence_len,), dtype='int32', name='words_input_mask')

words = Embedding(embedding.shape[0], embedding.shape[1], weights=[embedding], trainable=True, embeddings_regularizer=embeddings_reg, name="words_Embedding")(words_input)
words = Dropout(dropout_emb)(words)

output = Convolution1D(filters=256, kernel_size=3, activation=activation_fn, padding='same', strides=1, kernel_regularizer=kernel_reg)(words)
output = Dropout(dropout_model)(output)

output = Bidirectional( CuDNNGRU(units, return_sequences=True, recurrent_regularizer=recurrent_reg), merge_mode='concat') (output)
output_h = Activation('tanh') (output)

output1 = GlobalMaxPooling1D()(output_h) 
output2 = MaskMaxPoolingLayer()([output_h, words_input_mask]) 

output = Dense(1, kernel_regularizer=kernel_reg)(output_h)
output = Permute((2, 1))(output)
output = Activation('softmax', name="attn_softmax")(output)
output = Lambda(lambda x: tf.matmul(x[0], x[1])) ([output, output_h])
output3 = Flatten() (output)

output = Concatenate()([output1, output2, output3])
output = Dropout(dropout_pen)(output)

output = Dense(300, kernel_regularizer=kernel_reg, activation='relu')(output)
output = Dense(n_out, kernel_regularizer=kernel_reg)(output)
output = Activation('softmax')(output)

model = Model(inputs=[words_input, words_input_mask], outputs=[output])
model.compile(loss='sparse_categorical_crossentropy', optimizer=adadelta, metrics=['accuracy'])
model.summary(line_length=120)

In [ ]:
# Evaluation functions 

def get_precision(test_y_pred, test_y, label): 
    label_count = 0
    label_count_correct = 0
    
    for i in range(len(test_y_pred)):
        if test_y_pred[i] == label: 
            label_count += 1 
            if test_y_pred[i] == test_y[i]: 
                label_count_correct += 1
    
    if label_count_correct == 0: 
        return 0.0
    else: 
        ret = float(label_count_correct) / float(label_count)
        return ret
        

def get_macro_f1(test_y_pred, test_y, n_out):
    f1_sum = 0
    f1_count = 0
    for label in range(1, n_out):        
        prec = get_precision(test_y_pred, test_y, label)
        recall = get_precision(test_y, test_y_pred, label)
        f1 = 0 if float(prec+recall)==float(0) else float(2*prec*recall/float(prec+recall))
        f1_sum += f1
        f1_count += 1
    macro_f1 = float(f1_sum) / float(f1_count)    
    return macro_f1
    

def get_accuracy(test_y_pred, test_y):
    acc =  float(np.sum(test_y_pred == test_y)) / float(len(test_y))
    return acc
    
    
def predict_classes(prediction):
    return prediction.argmax(axis=-1)


def get_PRF1_semeval(y_pred, answer_key):
    
    f_out = open(proposed_ans, 'w')
    for i in range(len(y_pred)):
        f_out.write(str(i+1) + "\t" + int_to_label[y_pred[i]] + "\n" )
    f_out.close()
    
    if os.name == 'nt': 
        os.system(perl_exe_path + " " + scorer_perl_script + " " + proposed_ans + " " + answer_key + " > " + scorer_output)
    else: 
        os.system("perl " + " " + scorer_perl_script + " " + proposed_ans + " " + answer_key + " > " + scorer_output)
        
    f_in = open(scorer_output, 'r')
    lines = f_in.readlines()
    f_in.close()
    
    lines = [ l  for l in lines[-30:] if l.strip() != '']
        
    acc = float(lines[-17].strip().split()[-1][:-1]) / 100.0
    PRF1 = lines[-2].strip().split()
    P = float(PRF1[2][:-1]) / 100.0
    R = float(PRF1[5][:-1]) / 100.0
    macro_f1 = float(PRF1[8][:-1]) / 100.0
    
    return (acc, P, R, macro_f1)

In [ ]:
print("Start training... \n")

# train_x = train_x[:200]
# train_y = train_y[:200]
# train_x_copy = train_x_copy[:200]
# train_y_copy = train_y_copy[:200]
# val_x = val_x[:200]
# val_y = val_y[:200]
# test_x = test_x[:200]
# test_y = test_y[:200]


train_max_acc = 0
train_max_f1 = 0
val_max_acc = 0
val_max_f1 = 0
test_max_acc = 0
test_max_f1 = 0

test_f1_final = 0
test_f1_final_max = 0

es_epoch = 0 

for epoch in range(nb_epoch):
    print("Epoch: ", epoch+1, "/", nb_epoch)
    es_epoch += 1
    if es_epoch > es_epoch_stop:
        print("Early Stopping...") 
        break
    
    index = np.arange(len(train_x))
    np.random.shuffle(index)
    train_x = train_x[index]
    train_x_mask = train_x_mask[index]
    train_y = train_y[index]
    
    # Input lists for fit and predict
    #
    train_input_list = [train_x, train_x_mask]
    train_input_list_copy = [train_x_copy, train_x_mask_copy]
    val_input_list = [val_x, val_x_mask]
    test_input_list = [test_x, test_x_mask]
    #train_input_list = [train_x]
    #train_input_list_copy = [train_x_copy]
    #val_input_list = [val_x]
    #test_input_list = [test_x]

    
    model.fit(train_input_list, train_y, batch_size=batch_size, verbose=1, epochs=1) 
    
    train_y_pred = predict_classes(model.predict(train_input_list_copy, verbose=1))
    val_y_pred = predict_classes(model.predict(val_input_list, verbose=1))
    test_y_pred = predict_classes(model.predict(test_input_list, verbose=1))
    
    train_PRF1 = get_PRF1_semeval(train_y_pred, train_answer_key)
    val_PRF1 = get_PRF1_semeval(val_y_pred, val_answer_key)
    test_PRF1 = get_PRF1_semeval(test_y_pred, test_answer_key)

    train_max_f1 = max(train_max_f1, train_PRF1[3])
    val_max_f1 = max(val_max_f1, val_PRF1[3])
    test_max_f1 = max(test_max_f1, test_PRF1[3])

    train_max_acc = max(train_max_acc, train_PRF1[0])
    val_max_acc = max(val_max_acc, val_PRF1[0])
    test_max_acc = max(test_max_acc, test_PRF1[0])
    
    if val_max_f1 == val_PRF1[3]: 
        test_f1_final = test_PRF1[3]
        test_f1_final_max = max(test_f1_final, test_f1_final_max)
        if save_model : 
            model_file_name = "model.keras"
            model.save('./model/' + model_file_name)
            print("Model saved", model_file_name)
        es_epoch = 0
    
    print("Train Accuracy: %.4f (max: %.4f)" % (train_PRF1[0], train_max_acc))
    print("Val   Accuracy: %.4f (max: %.4f)" % (val_PRF1[0], val_max_acc))
    print("Test  Accuracy: %.4f (max: %.4f)" % (test_PRF1[0], test_max_acc))
    
    print("Train Macro F1 Semeval Official: %.4f (max: %.4f)" % (train_PRF1[3], train_max_f1))
    print("Val   Macro F1 Semeval Official: %.4f (max: %.4f)" % (val_PRF1[3], val_max_f1))
    print("Test  Macro F1 Semeval Official: %.4f (max: %.4f)" % (test_PRF1[3], test_max_f1))
    print("Test P %.4f | R %.4f | macro_F1: %.4f" % test_PRF1[1:] )
    print("Test test_max_f1_final: %.4f (max: %.4f)" % (test_f1_final, test_f1_final_max) )
        
    print()